# SiLA 2 Device Discovery

[SiLA 2](https://sila-standard.com/) is a communication standard used by many lab instruments. Instruments that speak SiLA 2 advertise themselves on the local network via mDNS (the `_sila._tcp.local.` service type), so you can find them without knowing their IP address upfront.

PyLabRobot provides a simple discovery function for this.

## Requirements

```bash
pip install zeroconf
```

Your computer must be on the same network (or VLAN) as the instruments you want to discover.

## Usage

In [ ]:
from pylabrobot.sila import discover

devices = await discover(timeout=5.0)
print(f"Found {len(devices)} device(s)")
for device in devices:
  print(device)

`discover()` listens for mDNS responses for `timeout` seconds and returns a list of `SiLADevice` objects. Each device has three attributes:

| Attribute | Type  | Description                     |
|-----------|-------|---------------------------------|
| `host`    | `str` | IP address (e.g. `192.168.1.42`)|
| `port`    | `int` | gRPC port (e.g. `8091`)         |
| `name`    | `str` | mDNS server name                |

## Example: connecting to the first device found

You can use the discovered host and port directly when creating a backend:

```python
from pylabrobot.sila import discover

devices = await discover()
assert len(devices) > 0, "No SiLA devices found"
device = devices[0]

# Pass device.host and device.port to your backend
backend = SomeBackend(host=device.host, port=device.port)
```

## Command-line tool

You can also discover devices from the terminal:

```bash
python -m pylabrobot.sila.discovery
```

Use `-t` to change the timeout (default 5 seconds):

```bash
python -m pylabrobot.sila.discovery -t 10
```

Output is tab-delimited (`host`, `port`, `name`), one device per line:

```
192.168.1.42	8091	ImageXpress-Pico._sila._tcp.local.
```